In [8]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import torch

In [2]:
check_point = 'encoder_decoder_pruned_last_3'
text = '근대 경제학에서는 행위의 합리성 문제를 비용 대 편익이라는 효율성 측면에서 접근하는데, 이러한 사고방식은 경제학을 넘어 사회과학 전반으로 널리 확산되 었다. 하지만 경제적 논리로는 설명하기 어렵거나 바람직하지 않은 결과를 낳은 현상도 많이 존재한다. 특히 도덕이나 규범은 경제 현상의 작동 방식에도 영향을 미치기 때문에 이를 설명하는 확장된 경제 이론이 필요하다. 도덕경제론은 바로 이러한 문제를 다루는 접근 방식이다. 이에 따르면 경제 활동에서 개인의 효용 극대화와 합리적 선택을 강조하는 경제학의 기본 가정 역시 초시대적으로 통용 되는 가치가 아니라 자본주의 등장 이후에 형성된 역사적 현상에 불과하다. 경제 적 원칙은 당대의 도덕적 규범에 의해 구성되는 상대적인 가치이기 때문에, 경제 활동이나 영역과 관련된 규범적 측면을 고려해 재구성해야 한다. ‘도덕경제 (moral economy)’론은 바로 이처럼 경제 현상에서 규범이나 문화의 역할 문제 를 다루는 접근 방식이다. 이러한 시도는 경제와 관련된 사회 현상을 이해하는 데 에서 경제학적 접근 방식의 편협성과 한계를 해결하는 데에도 풍부한 시사점을 줄수있을것이다. 이논문은기존의도덕경제론에서다룬주요쟁점과개념을 소개하고, 이 논의가 미디어 경제를 이해하는 데 주는 함의, 쟁점 등을 검토한다.'
tokenizer = AutoTokenizer.from_pretrained(check_point)
model = AutoModelForSeq2SeqLM.from_pretrained(check_point)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [3]:
input = tokenizer(text, return_tensors='pt')
label = tokenizer("임의로 생성한 라벨 역활", return_tensors='pt')

In [4]:
output = model(**input, labels=label['input_ids'], output_hidden_states=True, output_attentions=True)

In [47]:
torch.nn.functional.softmax(output.logits, dim=-1).mean()

tensor(3.3333e-05, grad_fn=<MeanBackward0>)

In [43]:
CE = torch.nn.CrossEntropyLoss()
CE(output.logits, output.logits)

tensor(-7.7680, grad_fn=<DivBackward1>)

In [5]:
len(output.decoder_hidden_states)

4

In [36]:
output.encoder_hidden_states[2].shape

torch.Size([1, 255, 768])

In [26]:
# 총 6개의 decoder attentions
loss = torch.nn.MSELoss()
loss(output.decoder_attentions[0], output.decoder_attentions[1])

tensor(0.0448, grad_fn=<MseLossBackward0>)

In [30]:
output.decoder_attentions[0][0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9054, 0.0946, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7747, 0.0464, 0.1788, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0972, 0.0380, 0.8451, 0.0197, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2796, 0.0808, 0.4112, 0.1547, 0.0736, 0.0000, 0.0000, 0.0000],
        [0.1933, 0.0358, 0.2729, 0.2479, 0.1858, 0.0642, 0.0000, 0.0000],
        [0.2333, 0.0452, 0.1265, 0.1575, 0.2720, 0.1139, 0.0515, 0.0000],
        [0.1248, 0.0282, 0.2672, 0.0834, 0.1690, 0.1697, 0.1070, 0.0506]],
       grad_fn=<SelectBackward0>)

In [29]:
loss_sum = 0
for i in range(len(output.decoder_attentions[0][0])):
    loss_sum += loss(output.decoder_attentions[0][0][i], output.decoder_attentions[1][0][i])
print(loss_sum/len(output.decoder_attentions[0][0]))

tensor(0.0448, grad_fn=<DivBackward0>)


In [ ]:
output.decoder_attentions[0].shape

torch.Size([1, 16, 8, 8])

In [7]:
model.model

BartModel(
  (shared): Embedding(30000, 768, padding_idx=3)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(30000, 768, padding_idx=3)
    (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
    (layers): ModuleList(
      (0): BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj)